# Detect data bias with Amazon SageMaker Clarify

## Import libraries

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.35.0
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

Create the boto3 client sdk that will be passed to sagemaker session as sagemaker client.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import botocore

config = botocore.config.Config(user_agent_extra='dlai-pds/c1/w2')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', 
                  config=config)

sess = sagemaker.Session(sagemaker_client=sm)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

In [ ]:
sess, bucket, role, region

## Analyze the dataset

Create a pandas data frame from the transformed/preprocessed CSV file.

In [ ]:
!aws s3 cp 's3://dlai-practical-data-science/data/transformed/womens_clothing_ecommerce_reviews_transformed.csv' ./

In [ ]:
path = './womens_clothing_ecommerce_reviews_transformed.csv'

df = pd.read_csv(path)
df.head()

In [ ]:
import seaborn as sns

sns.countplot(data=df, x='sentiment', hue='product_category')

plt.legend(loc='upper right',bbox_to_anchor=(1.3, 1.1))

## Upload the dataset to S3 bucket

In [ ]:
data_s3_uri_unbalanced = sess.upload_data(bucket=bucket, 
                               key_prefix='bias/unbalanced', 
                               path='./womens_clothing_ecommerce_reviews_transformed.csv')
data_s3_uri_unbalanced

## Analyze class imbalance on the dataset with Amazon SageMaker Clarify

Let's analyze bias in sentiment with respect to the product_category facet on the dataset.

### Configure a DataConfig

Information about the input data needs to be provided to the processor. This can be done with the DataConfig of the Clarify container. It stores information about the dataset to be analyzed, for example the dataset file, its format, headers and labels.

In [ ]:
from sagemaker import clarify

bias_report_unbalanced_output_path = 's3://{}/bias/generated_bias_report/unbalanced'.format(bucket)

data_config_unbalanced = clarify.DataConfig(
    s3_data_input_path=data_s3_uri_unbalanced, # Replace None
    s3_output_path=bias_report_unbalanced_output_path, # Replace None
    label='sentiment', # Replace None
    headers=df.columns.to_list(),
    dataset_type='text/csv'
)

### Configure BiasConfig

In [ ]:
bias_config_unbalanced = clarify.BiasConfig(
    label_values_or_threshold=[1], # desired sentiment
    facet_name='product_category' # sensitive column (facet)
)

Bias is measured by calculating a metric and comparing it across groups. To compute it, you will specify the required information in the BiasConfig API. SageMaker Clarify needs the sensitive columns (facet_name) and the desirable outcomes (label_values_or_threshold). Here product_category is the sensitive facet and the desired outcome is with the sentiment==1.

SageMaker Clarify can handle both categorical and continuous data for label_values_or_threshold. In this case you are using categorical data

## Configure Amazon SageMaker Clarify as a processing job

Now you need to construct an object called SageMakerClarifyProcessor. This allows you to scale the process of data bias detection using two parameters, instance_count and instance_type. Instance_count represents how many nodes you want in the distributor cluster during the data detection. Instance_type specifies the processing capability (compute capacity, memory capacity) available for each one of those nodes. 

In [ ]:
clarify_processor_unbalanced = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.large',
                                                      sagemaker_session=sess)

## Run the Amazon SageMaker Clarify processing job

Run the configured processing job to compute the requested bias methods of the input data

In [ ]:
clarify_processor_unbalanced.run_pre_training_bias(
    data_config=data_config_unbalanced, # Replace None
    data_bias_config=bias_config_unbalanced, # Replace None
    methods=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"],
    wait=False,
    logs=False
)

View the job run

In [ ]:
run_unbalanced_bias_processing_job_name = clarify_processor_unbalanced.latest_job.job_name
print(run_unbalanced_bias_processing_job_name)

## Run and review the Amazon SageMaker Clarify processing job on the unbalanced dataset

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=run_unbalanced_bias_processing_job_name,
                                                                            sagemaker_session=sess)

## Analyze balanced bias report

In [ ]:
!aws s3 ls $bias_report_balanced_output_path/

### Download generated bias report from S3 bucket:

In [ ]:
!aws s3 cp --recursive $bias_report_balanced_output_path ./generated_bias_report/balanced/